In [ ]:
import os
import time

import numpy as np
import tensorflow as tf

In [ ]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)
#download file

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")
#check length of text

Length of text: 1115394 characters


In [ ]:
print(text[:250])
#print first 250 characters in text

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")
#how many unique letters we have

65 unique characters


In [ ]:
example_texts = ["abcdefg", "xyz"]

# TODO 1
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)
#create the tf.keras.layers.StringLookup layer:

In [ ]:
ids = ids_from_chars(chars)
ids
#convert tokens to char ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [ ]:
chars = chars_from_ids(ids)
chars
#join characters back to string

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
#prediction
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:

split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [28]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
sampled_indices
#give us at each timestep a prediction of the nexr character index

array([43,  0, 51, 18, 28, 52, 37, 11,  7,  8, 65, 45, 19, 23, 47, 18, 52,
        7, 61, 46, 45, 49, 36, 34, 36,  0,  4, 41, 55, 25, 38, 48, 36, 56,
       56, 31, 24,  9, 59, 17, 47, 27, 45, 29, 52, 40, 49, 23, 14,  5, 17,
       27, 29, 48, 40, 24, 61, 17, 14,  1, 26, 59, 59, 26, 43, 50, 41, 38,
       24, 22, 54, 17,  1, 25, 27, 47,  3, 58, 53, 59, 48, 25, 34, 20, 30,
       32, 41, 37, 15,  2,  7, 43, 21, 20, 21, 14, 49, 35, 62, 14])

In [35]:
#Decode these to see the text predicted by this untrained model
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'-tongued wife,\nWhom for this time we pardon. We enjoin thee,\nAs thou art liege-man to us, that thou '

Next Char Predictions:
 b'd[UNK]lEOmX:,-zfFJhEm,vgfjWUW[UNK]$bpLYiWqqRK.tDhNfPmajJA&DNPiaKvDA\nMttMdkbYKIoD\nLNh!sntiLUGQSbXB ,dHGHAjVwA'


In [36]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189247, shape=(), dtype=float32)


In [38]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189247, shape=(), dtype=float32)


In [39]:
model.compile(optimizer="adam", loss=loss)

In [40]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [41]:
EPOCHS = 10
#use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.
#to get better results train the model more epochs=30

In [42]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
  7/172 [>.............................] - ETA: 27:14 - loss: 4.4088

KeyboardInterrupt: ignored

In [43]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
               # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [44]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [45]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:R$pXDtuYs,IJSOWBKm:j&!: -!3mgjpb'O srw
m gDrH.qmkE,k3EKe  -gRjgQI.H.v.aTezTE;LwrDHxCn .B:GzK,N;tk!lL.ZJVeRt?umPg-sPumFeG bzwSxsYfd$y'r;x
VUEGYp
,Q!&pXUjAFTfleQADQV$aN.&.BcvocMcDqivEF
qhiVRYtz3lQU$zAREPt -Hxu'MA'UvmB ROaadAfl:xq;VwlTUFpzCDcJsj;Z-apBR3yCUYv V$:IBd-xajEgkdMYfPR!uAfcn$M$af3RhNJ'uYy K'Ki;'Wlr:Hs,PulTdcwtxBs?MgVilev3jdSFDx&,&R$SCgVrbLdbNrrq 
ZPweV&'pgbCCGeC;Aw-

FFIQ.uZHu KifNackiNjgebVUvmSlVDZnpsa,
C,ScirsfSVMQisMwAOAVy.$j
' hZAolSROhS.
Sep$A:WS!?.OnWoiqhDH$ZsSGEN!ebRe.sxXuJwwhRVYttUJMT'JeBAjnEmNWRBHUNHohC,wkhAze$cOwAbgvV:L
QKYmd:UlUBKQY$RI&XzVvzXFxyEtPEnQxsBScf'gqER&qZTNkRgcOQE3pXamS&GE-RVgd,$ofvLc .TNRl3Rb VJ&uJymZHMmyd.sAcrNeGpS!Ne,deOHGnh?UiZJXUU;m
Bo$XeX!GSZ;3
bb;pMJiMR'mPOD&oOqQer.:u3HOgnROa3PhBO&AzrYxIywA'&&G
ftTdv;ucZD
bVKimc3mQfTh
I!-M:f se? 3piPEipOeE.VoSRCq;EY!.rp,Yqk;N&F-zreObfcXFOBTL!hxLN: tI!pxN.l-y,-.sK!C:qcyiAK
H
xYF.znbspoe;ens3wv -n.:s;tnoTwclZ:ZeR$ht& FRALeiG$ikAy;t,yyvn-e.rtF3&& JioDACh3i- '?rX wd'LcEnQ j ';m;-WozT$OLEgJ,?,tEGdgvjdEp;AJYGaZMbtCPkKb

In [46]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, "\n\n" + "_" * 80)
print("\nRun time:", end - start)

tf.Tensor(
[b"ROMEO:umM.PeaMxhdgB;WKcwI?DdpmJysQamcIDv rA;cQ- KsS,cmqC'K-YsE, k:ivv&,?!-?yVyFArx'k\nNZ&PD$vj.L\nH-hlPSZMIobZha;hbJLtgTjGDI-lu.:b,q'hkDQ'yvlQaF EDUG\nZ&StYKvphQm3iZnUrPfTks;As,$IezG,xrwS.E\nL,?yUSRteoOlkF',\n\no.nZVvPv'VRG'X$-r\nAwHLQg$xjdsfEzt,sp$hI3k$fBNrx\nVQVmRNZslxKlKUewDD;F;K&WxybDbNOYiOnKShGAI;,BBLzWqQpnSuD;!yr-v&lyC:hybusH,OqcH3WH$LNd?A\ngGEPtQLyiRy3O.kbwdHhor3ddCoDA'tnCPWd Jr&SHxZ?'AnwcL3l!-:?oehwPD-'LdQbYu\n&bxv-;:PLls;-Vh$W'YBvuQFu,q YLoA\n;cmNhcbfdMVgmyx.GzznB-!!yG;iMe.iUMEA.Pbot WTxkfeqmhVi&$orTeQb.eYaHy&WpNWR.M.Q-:eNpo!!uULtnctr. QCqhFet!T!M!XRQGUqLFXxX?bAVdQ!b-EG\nzLJc;v-HgB!v.lWOAEOxi-!MjJE;:FKCxOWQzH.-OObSSm,-wSJBoUrKl.UZy!'uBKinnQWwgTcEH;QwVsl.o'RCKFbSMKKJmIXW$XCoGD!ocD'.-XXto-v&W'zdvvBrMx&?VLSXtf3iC-N:IxQd'BSV.k\nWM.sasw;L,?sjJv:GCxUK$uHbViJgesb-JqQ;nl?$W?:ku!B-B-\nC DiMsYeTIT:tcKvM!Ic\n!N'bUio\n!$RFt!?$\npMBtACO.Scnz-Cum$?Gq$Rg.\nygR?qsO!A$VJmd.ozk;j\nNw;p$,o-OeLUNTahBU$iDn!s' ?.jznnoWf.AFbp3?Z&dzU'oQOWfs s\nGcy\nRw:xsLmnwsiuqhWUmB'R.hXD 3W:ADMIl!iEq'

In [47]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [48]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:nVUDmkkl:XxwI'mkjVXs,BQH Gzn:hdJoJPqduabBf-sUyDiO$IEmYSLfGhACa'd
qSvr:RQ'tE-.IETiQAjaXa,yAVxKf HXUi,
